In [ ]:
nan

# Import Necessary Libraries

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt

import cv2
from tqdm import tqdm
import io
import seaborn as sns

from sklearn.utils import shuffle # Shuffle arrays or sparse matrices in a consistent way
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.applications import EfficientNetB0
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard,ModelCheckpoint

from sklearn.metrics import classification_report, confusion_matrix
from keras.utils.vis_utils import plot_model

import ipywidgets as widgets
from PIL import Image
from IPython.display import display, clear_output

# Labels in order of severity

In [ ]:
labels = ['CONTROL', 'AD', 'PD']

# What is tqdm?
<br><br>
1. tqdm is a Python library that allows you to output a smart progress bar by wrapping around any iterable. 
<br><br>
2. A tqdm progress bar not only shows you how much time has elapsed, but also shows the estimated time remaining for the iterable.
<br><br>
3. Resulting tqdm progress bar gives us information that includes the task completion percentage, number of iterations complete, time elapsed, estimated time remaining, and the iterations completed per second.


In [ ]:
X_train = [] #Training Dataset
Y_train = [] #Training Labels

image_size=150

for i in labels:
    folderPath = os.path.join('../input/alzheimer-diseases-3-class/3_cls/', 'train', i)
    for j in tqdm(os.listdir(folderPath)):
        image = cv2.imread(os.path.join(folderPath, j))
        image = cv2.resize(image, (image_size, image_size))
        X_train.append(image)
        Y_train.append(i)
        
        
for i in labels:
    folderPath = os.path.join('../input/alzheimer-diseases-3-class/3_cls/', 'test', i) # Join two or more pathname components
    for j in tqdm(os.listdir(folderPath)):
        image = cv2.imread(os.path.join(folderPath, j))
        image = cv2.resize(image, (image_size, image_size))
        X_train.append(image)
        Y_train.append(i)
        
#Image and Label is appended as list, now is to be converted into array
X_train = np.array(X_train)
Y_train = np.array(Y_train)

In [ ]:
X_train.shape #No of sample = 7839

In [ ]:
X_train, Y_train = shuffle(X_train, Y_train, random_state=42)

In [ ]:
#After shuffling sample size remains same
X_train.shape

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.1, random_state=42)

# Converting String Label to categorical

In [ ]:
y_train_new = []
y_test_new = []

for i in Y_train:
    y_train_new.append(labels.index(i))#Converting String Label to integer i.e
                                       # CONTROL ---> 0, AD---> 1, PD ---> 2
Y_train = to_categorical(y_train_new) #Converts a class vector (integers) to binary class matrix

for i in Y_test:
    y_test_new.append(labels.index(i))

Y_test = to_categorical(y_test_new)

In [ ]:
Y_train

![image.png](attachment:dbe6dd77-bbb7-4233-b64d-833cf8ed7932.png)

In [ ]:
# Input of model
efficientnet_B0 = EfficientNetB0(include_top=False,
    weights='imagenet',
    input_shape=(image_size, image_size, 3))

In [ ]:
# Output of model
model = efficientnet_B0.output
model = tf.keras.layers.GlobalAveragePooling2D()(model)
model = tf.keras.layers.Dropout(0.5)(model)
model = tf.keras.layers.Dense(3, activation='softmax')(model)

In [ ]:
#Merge input and Output of model
model = tf.keras.models.Model(inputs=efficientnet_B0.input, outputs=model)

model.summary()

In [ ]:
#Configures the model for training
model.compile(optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['acc'])

# TensorBoard
![image.png](attachment:6abbe112-ae3d-4ada-8c43-d45f4a010b2f.png)

# ModelCheckpoint
![image.png](attachment:740791f4-85d2-4f87-a42e-977e08e62a4d.png)

# Verbose
verbose: 'auto', 0, 1, or 2. Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch. 'auto' defaults to 1 for most cases

# ReduceLROnPlateau
![image.png](attachment:508a3aa0-7a37-402a-a96b-c9f63c7728de.png)
<br>
1. Models often benefit from reducing the learning rate by a factor
of 2-10 once learning stagnates. 
<br>
2. This callback monitors a quantity and if no improvement is seen for a 'patience' number
of epochs, the learning rate is reduced.
<br>
3. monitor: quantity to be monitored.
<br>
4. factor: factor by which the learning rate will be reduced.
<br>
       <b> new_lr = lr * factor`. </b>
<br>
5. patience: number of epochs with no improvement after which learning rate
      will be reduced.
<br>
6. min_delta: threshold for measuring the new optimum, to only focus on
      significant changes.

# EarlyStopping
![image.png](attachment:e4fadc60-f3ba-4c74-9c94-33887ac7478f.png)

In [ ]:
tensorBoard = TensorBoard(log_dir="logs")
checkPoint = ModelCheckpoint("efficient_net_B0.h5",
                            monitor='val_loss',
                            verbose=1,
                            save_best_only=True,
                            mode='auto')
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                             factor=0.3,
                             patience=2,
                             verbose=1,
                             mode='auto',
                             min_delta=0.001)
es = EarlyStopping(monitor='val_loss',
                   patience=5,
                   verbose=1,
                   mode='auto',
                   restore_best_weights=True) 

In [ ]:
history = model.fit(X_train, 
                    Y_train,
                    batch_size=32,
                    validation_split=0.1,
                    epochs=20,
                    verbose=1,
                    callbacks=[tensorBoard, checkPoint, reduce_lr, es])

![image.png](attachment:4b2dd021-61aa-4f29-af5c-897bba70c11b.png)

In [ ]:
pred = model.predict(X_test)
pred = np.argmax(pred, axis=1)
actual_label = np.argmax(Y_test, axis=1)
print(classification_report(actual_label, pred))

In [ ]:
cnf = confusion_matrix(actual_label, pred)
plt.figure(figsize=(8,6), dpi=70, facecolor='w', edgecolor='k')
ax = sns.heatmap(cnf, cmap='Blues',annot=True, fmt='d', xticklabels=labels, yticklabels=labels)
plt.title('Alzheimer\'s Disease Classification')
plt.xlabel('Prediction')
plt.ylabel('Ground Truth')
plt.show(ax)

# plot_model
![image.png](attachment:75c53aec-61f8-4d39-9b60-c644c0ae457e.png)

In [ ]:
saved_model = tf.keras.models.load_model("./efficient_net_B0.h5")

#Show Model Architecture
plot_model(saved_model,
          to_file='efficient_net_B0.png',
          show_shapes=True,
          show_layer_names=True)

# Uploading Image and Showing its Class

1. BytesIO is used to read locally stored image
2. FileUpload() ---> Upload file(s) from browser to Python kernel as bytes
3. <b>display</b>
<br>
    a. Display a Python object in all frontends.
<br>
    b. By default all representations will be computed and sent to the frontends.
<br>
    c. Frontends can decide which representation is used and how.
4. <b>Button widget</b>
   <br>
   This widget has an `on_click` method that allows you to listen for the user clicking on the button
5. <b> Output </b> 
   <br>
Widget used as a context manager to display output
6. <b> clear_output() </b>
   <br>
   Clear the output of the current cell receiving output
7. <b> on_click()</b>
<br>
Register a callback to execute when the button is clicked.
8. <b> VBox</b>
<br>
Displays multiple widgets vertically using the flexible box model

In [ ]:
def imagePrediction(upload):
    for name, fileinfo  in uploader.value.items():
        image = Image.open(io.BytesIO(fileinfo['content']))
        
    images = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    images = cv2.resize(images,(150, 150))
    images = images.reshape(1, 150, 150, 3)
    prd = model.predict(images)
    prd = np.argmax(prd, axis = 1)[0]
    
    
    if prd == 0:
        prd = "CONTROL"
    elif prd == 1:
        prd = "AD"
    elif prd ==2:
        prd = "PD"
        
    print(f'Model Predict That is  a {prd}')

In [ ]:
uploader = widgets.FileUpload()
display(uploader)

In [ ]:
button = widgets.Button(description= "Predict")
out = widgets.Output()

def on_button_click(_):
    with out:
        clear_output()
        try:
            imagePrediction(uploader)
        except:
            print("Please Enter the Correct Image files")
            
            
button.on_click(on_button_click)
widgets.VBox([button, out])

In [ ]:
nan